In [1]:
import json
import scipy.sparse as ss
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import scipy.io as sio

np.random.seed(123)

In [2]:
foil_train = json.load(open('foil_new/FOIL-LINGv0.5_train2017.json'))
ourimagetrainfeats = [(int(w.strip().split()[0].split('.jpg')[0].split('COCO_train2014_')[1]), np.array(map(float, w.strip().split()[1:]))) 
                      for w in open('data/mscoco_boc_gt_train2014.txt')]
ourimagetrainfeats = dict(ourimagetrainfeats)
training_annotations = [l['caption'] for l in foil_train['annotations']]

tf_vectorizer = CountVectorizer(max_features=None, lowercase=True)
tf_model = tf_vectorizer.fit(training_annotations)
training_feats = tf_model.transform(training_annotations)

training_y = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_train['annotations']]
training_image_feats = [ourimagetrainfeats[i['image_id']] for i in foil_train['annotations']]
training_image_feats_sparse = ss.csr_matrix(np.array(training_image_feats))
print len(training_image_feats)
foil_test = json.load(open('foil_new/FOIL-LINGv0.5_test2017.json') )
testing_annotations = [l['caption'] for l in foil_test['annotations']]
ourimagetestfeats = [(int(w.strip().split()[0].split('.jpg')[0].split('COCO_val2014_')[1]),
                     map(float, w.strip().split()[1:])) 
                    for w in open('data/mscoco_boc_gt_val2014.txt')]
ourimagetestfeats = dict(ourimagetestfeats)

testing_annotations = [l['caption'] for l in foil_test['annotations']]
testing_feats = tf_model.transform(testing_annotations) 
testing_y = [0 if f['foil_word'] == 'ORIG' else 1 for f in foil_test['annotations']]                    
testing_image_feats = [ourimagetestfeats[i['image_id']] for i in foil_test['annotations']]
testing_image_feats_sparse = ss.csr_matrix(np.array(testing_image_feats))

106691


In [5]:
def write_stuff(annots, ys, filename):
    with open(filename, 'w') as f:
        for x,y in zip(annots, ys):
            f.write('%s\t%s\n' % (y, x.replace('\n', ' ').strip('\n').strip()))
            
    print 'done'
    
    
write_stuff(training_annotations, training_y, 'POSTraining.txt')
write_stuff(testing_annotations, testing_y, 'POSTesting.txt')


from tqdm import tqdm

def dump(dirtrain, dirtest, dtypetr, dtypete):
    for r in tqdm(range(len(dtypetr))):
        np.save(dirtrain + '/' + str(r+1)+'.npy', dtypetr[r])
    print 'done with training'
    for r in tqdm(range(len(dtypete))):
        np.save(dirtest + '/' + str(r+1)+'.npy', dtypete[r])
    print 'done with testing'
    
    
%mkdir train_img_pos test_img_pos
dump('train_img_pos', 'test_img_pos',  training_image_feats, testing_image_feats)


done
done


  2%|▏         | 1135/61746 [00:00<00:05, 11346.14it/s]

done with training


100%|██████████| 61746/61746 [00:05<00:00, 11179.22it/s]

done with testing


In [7]:
sio.mmwrite('data_new/train_feats.mtx', training_feats)
sio.mmwrite('data_new/test_feats.mtx', testing_feats)
sio.mmwrite('data_new/train_image_feats.mtx', training_image_feats_sparse)
sio.mmwrite('data_new/test_image_feats.mtx', testing_image_feats_sparse)
sio.mmwrite('data_new/train_y.mtx', ss.csr_matrix(np.array(training_y)))
sio.mmwrite('data_new/test_y.mtx', ss.csr_matrix(np.array(testing_y)))

In [8]:
foil_train['annotations'][3]

{u'POS': u'PP',
 u'Source': u'VerbOcean',
 u'caption': u'a group of men below a table preparing food together',
 u'foil_id': 4000007,
 u'foil_word': u'below',
 u'id': 486903,
 u'image_id': 405613,
 u'target_word': u'at'}

In [9]:
foiled_stuff = [a for a in foil_train['annotations'] if a['foil_word'] != 'ORIG' ]

In [12]:
m = [a['POS'] for a in foiled_stuff]
set(m)



{u'ADJ', u'ADV', u'PP', u'VERB'}

In [4]:
len([s for s in foil_train['annotations'] if s['foil_word'] != 'ORIG'])

54354